In [1]:
import pandas as pd
import numpy as np

In [9]:
# FOR UNTIL JULY

df_oil = pd.read_excel('./data-july/oil_front_month.xlsx', sheet_name='Part 0', skiprows=[0, 1, 2], index_col=0)
df_gas = pd.read_csv('./data-july/ttf_da.csv', skiprows=[0, 1, 2, 3, 4, 5], index_col=0)
df_gas.columns = ['ttf']
df_coal = pd.read_excel('./data-july/api2_front_month.xlsx', sheet_name='Part 0', skiprows=[0, 1, 2], index_col=0)
df_currency = pd.read_excel('./data-july/currency.xlsx', sheet_name='Part 0', skiprows=[0, 1, 2], index_col=0)

df_demand = pd.read_excel("./data-july/demand.xlsx", index_col=0)

In [10]:
start_date = np.datetime64("2022-08-31") #pd.DateTime('20220901', tz='Europe/Amsterdam')
end_date = np.datetime64("2023-07-30") #pd.DateTime('20230228', tz='Europe/Amsterdam')

df_oil.index = pd.to_datetime(df_oil.index)
df_gas.index = pd.to_datetime(df_gas.index)
df_coal.index = pd.to_datetime(df_coal.index)
df_currency.index = pd.to_datetime(df_currency.index)

df_oil = df_oil.loc[(df_oil.index > start_date) & (df_oil.index <= end_date)]
df_gas = df_gas.loc[(df_gas.index > start_date) & (df_gas.index <= end_date)]
df_coal = df_coal.loc[(df_coal.index > start_date) & (df_coal.index <= end_date)]
df_currency = df_currency.loc[(df_currency.index > start_date) & (df_currency.index <= end_date)]

display(df_oil)
display(df_gas)
display(df_coal)
display(df_currency)

,USD 12 Months,USD
Date,,
2022-09-01,78.79,92.36
2022-09-02,78.79,93.02
2022-09-03,NaN,NaN
2022-09-04,NaN,NaN
2022-09-05,78.97,95.74
...,...,...
2023-07-26,78.73,82.92
2023-07-27,79.60,84.24
2023-07-28,80.16,84.99


,ttf
Date,
2022-09-01,237.050
2022-09-02,225.050
2022-09-03,NaN
2022-09-04,NaN
2022-09-05,180.150
...,...
2023-07-24,27.175
2023-07-25,29.850
2023-07-26,32.225


,Front Year,Front Month
Date,,
2022-09-01,312.00,358.00
2022-09-02,323.00,367.00
2022-09-03,NaN,NaN
2022-09-04,NaN,NaN
2022-09-05,344.00,394.00
...,...,...
2023-07-26,127.55,115.00
2023-07-27,126.73,113.50
2023-07-28,119.50,106.75


,EURO to USD
Date,
2022-09-01,1.00
2022-09-02,1.00
2022-09-03,1.00
2022-09-04,1.00
2022-09-05,0.99
...,...
2023-07-26,1.11
2023-07-27,1.11
2023-07-28,1.10


In [12]:
df_fuel = pd.DataFrame()
df_fuel.index = df_oil.index
df_fuel['coal'] = df_coal['Front Month'] / df_currency['EURO to USD']
df_fuel['oil'] = df_oil['USD'] / df_currency['EURO to USD']
df_fuel['gas'] = df_gas['ttf']

for index, row in df_fuel.iterrows():
    if np.isnan(row['coal']):
        loc = df_fuel.index.get_loc(index)
        df_fuel.loc[index, 'coal'] = df_fuel.iloc[loc - 1]['coal']

    if np.isnan(row['gas']):
        loc = df_fuel.index.get_loc(index)
        df_fuel.loc[index, 'gas'] = df_fuel.iloc[loc - 1]['gas']
        
    if np.isnan(row['oil']):
        loc = df_fuel.index.get_loc(index)
        df_fuel.loc[index, 'oil'] = df_fuel.iloc[loc - 1]['oil']

display(df_fuel)

,coal,oil,gas
Date,,,
2022-09-01,358.000000,92.360000,237.050
2022-09-02,367.000000,93.020000,225.050
2022-09-03,367.000000,93.020000,225.050
2022-09-04,367.000000,93.020000,225.050
2022-09-05,397.979798,96.707071,180.150
...,...,...,...
2023-07-26,103.603604,74.702703,32.225
2023-07-27,102.252252,75.891892,28.625
2023-07-28,97.045455,77.263636,28.150


In [13]:
df_fuel_expanded = pd.DataFrame()
df_fuel_expanded.index = df_demand.index

df_fuel_expanded.index = pd.to_datetime(df_fuel_expanded.index)

for index, row in df_fuel_expanded.iterrows():
    df_fuel_expanded.loc[index, 'coal'] = df_fuel.loc[index.replace(hour=0, minute=0), 'coal']
    df_fuel_expanded.loc[index, 'oil'] = df_fuel.loc[index.replace(hour=0, minute=0), 'oil']
    df_fuel_expanded.loc[index, 'gas'] = df_fuel.loc[index.replace(hour=0, minute=0), 'gas']
    
df_fuel_expanded.to_excel("./data-july/fuel_prices.xlsx")

In [2]:
df_oil = pd.read_excel('./data/oil_month_ahead.xlsx', sheet_name='Part 0', skiprows=[0, 1, 2], index_col=0)
df_gas_coal = pd.read_excel('./data/fuel_history.xlsx', sheet_name='Sheet1', index_col=0)
df_currency = pd.read_excel('./data/currency.xlsx', sheet_name='Part 0', skiprows=[0, 1, 2], index_col=0)

df_demand = pd.read_excel("./data/demand.xlsx", index_col=0)

In [3]:
start_date = np.datetime64("2022-08-31") #pd.DateTime('20220901', tz='Europe/Amsterdam')
end_date = np.datetime64("2023-04-01") #pd.DateTime('20230228', tz='Europe/Amsterdam')

df_oil.index = pd.to_datetime(df_oil.index)
df_gas_coal.index = pd.to_datetime(df_gas_coal.index)
df_currency.index = pd.to_datetime(df_currency.index)

df_oil = df_oil.loc[(df_oil.index > start_date) & (df_oil.index <= end_date)]
df_gas_coal = df_gas_coal.loc[(df_gas_coal.index > start_date) & (df_gas_coal.index <= end_date)]
df_currency = df_currency.loc[(df_currency.index > start_date) & (df_currency.index <= end_date)]

display(df_oil)
display(df_gas_coal)
display(df_currency)

,USD 12 Months,USD
Date,,
2022-09-01,80.58,92.36
2022-09-02,80.57,93.02
2022-09-03,NaN,NaN
2022-09-04,NaN,NaN
2022-09-05,81.10,95.74
...,...,...
2023-03-28,75.21,78.65
2023-03-29,74.73,78.28
2023-03-30,75.49,79.27


,Week,EPEX NL Base,EPEX NL Peak,CO2,TTF DA,API2,EUR/USD
Date,,,,,,,
2022-09-01,35,534.68,539.75,81.5,268.00,360.00,1.0013
2022-09-02,35,425.28,427.77,78.5,230.00,355.00,1.0053
2022-09-03,35,286.35,NaN,81.0,205.00,366.25,0.9944
2022-09-04,35,250.69,NaN,78.0,200.00,366.25,0.9944
2022-09-05,36,359.16,376.01,78.0,180.00,366.25,0.9944
...,...,...,...,...,...,...,...
2023-03-27,13,81.09,51.29,87.8,42.00,125.00,1.0827
2023-03-28,13,114.14,112.43,89.0,42.60,133.00,1.0765
2023-03-29,13,116.18,124.41,88.9,41.70,139.50,1.0797


,EURO to USD
Date,
2022-09-01,1.00
2022-09-02,1.00
2022-09-03,1.00
2022-09-04,1.00
2022-09-05,0.99
...,...
2023-03-28,1.08
2023-03-29,1.08
2023-03-30,1.09


In [6]:
df_fuel = pd.DataFrame()
df_fuel.index = df_oil.index
df_fuel['coal'] = df_gas_coal['API2'] / df_currency['EURO to USD']
df_fuel['oil'] = df_oil['USD'] / df_currency['EURO to USD']
df_fuel['gas'] = df_gas_coal['TTF DA']
df_fuel['eua'] = df_gas_coal['CO2']

for index, row in df_fuel.iterrows():
    if np.isnan(row['coal']):
        loc = df_fuel.index.get_loc(index)
        df_fuel.loc[index, 'coal'] = df_fuel.iloc[loc - 1]['coal']
        
    if np.isnan(row['oil']):
        loc = df_fuel.index.get_loc(index)
        df_fuel.loc[index, 'oil'] = df_fuel.iloc[loc - 1]['oil']
        
    if np.isnan(row['eua']):
        loc = df_fuel.index.get_loc(index)
        df_fuel.loc[index, 'eua'] = df_fuel.iloc[loc - 1]['eua']

display(df_fuel)

,coal,oil,gas,eua
Date,,,,
2022-09-01,360.000000,92.360000,268.00,81.5
2022-09-02,355.000000,93.020000,230.00,78.5
2022-09-03,366.250000,93.020000,205.00,81.0
2022-09-04,366.250000,93.020000,200.00,78.0
2022-09-05,369.949495,96.707071,180.00,78.0
...,...,...,...,...
2023-03-28,123.148148,72.824074,42.60,89.0
2023-03-29,129.166667,72.481481,41.70,88.9
2023-03-30,126.605505,72.724771,43.75,90.0


In [7]:
df_fuel_expanded = pd.DataFrame()
df_fuel_expanded.index = df_demand.index

df_fuel_expanded.index = pd.to_datetime(df_fuel_expanded.index)

for index, row in df_fuel_expanded.iterrows():
    df_fuel_expanded.loc[index, 'coal'] = df_fuel.loc[index.replace(hour=0, minute=0), 'coal']
    df_fuel_expanded.loc[index, 'oil'] = df_fuel.loc[index.replace(hour=0, minute=0), 'oil']
    df_fuel_expanded.loc[index, 'gas'] = df_fuel.loc[index.replace(hour=0, minute=0), 'gas']
    df_fuel_expanded.loc[index, 'eua'] = df_fuel.loc[index.replace(hour=0, minute=0), 'eua']
    
df_fuel_expanded.to_excel("./data/fuel_prices.xlsx")